In [274]:
import sqlite3

# SQLite データベースファイルのパス
DB_FILE = "weather.db"

def create_tables():
    """必要なテーブルを作成する"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # regionsテーブル
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS regions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        )
        """)

        # prefecturesテーブル
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS prefectures (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            region_id INTEGER,
            FOREIGN KEY (region_id) REFERENCES regions (id)
        )
        """)

        # areasテーブル
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS areas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            prefecture_id INTEGER,
            FOREIGN KEY (prefecture_id) REFERENCES prefectures (id)
        )
        """)

        # weatherテーブル
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            area_id INTEGER,
            date_time TEXT,
            weather TEXT,
            wind TEXT,
            wave TEXT,
            FOREIGN KEY (area_id) REFERENCES areas (id)
        )
        """)

        conn.commit()
    except sqlite3.Error as e:
        print(f"テーブル作成エラー: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    create_tables()  # テーブルを作成


In [275]:
import sqlite3
import requests
import json

# SQLite データベースファイルのパス
DB_FILE = "weather.db"
FORECAST_API_URL = "https://www.jma.go.jp/bosai/forecast/data/forecast/{region_code}.json"

def save_region_to_db(region_name):
    """地方名をregionsテーブルに保存"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("SELECT id FROM regions WHERE name = ?", (region_name,))
        region_id = cursor.fetchone()

        if not region_id:
            cursor.execute("INSERT INTO regions (name) VALUES (?)", (region_name,))
            conn.commit()
            region_id = cursor.lastrowid
        else:
            region_id = region_id[0]

        return region_id
    except sqlite3.Error as e:
        print(f"データベースエラー: {e}")
        return None
    finally:
        conn.close()

def save_prefecture_to_db(prefecture_name, region_id):
    """県名をprefecturesテーブルに保存"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("SELECT id FROM prefectures WHERE name = ?", (prefecture_name,))
        prefecture_id = cursor.fetchone()

        if not prefecture_id:
            cursor.execute(
                "INSERT INTO prefectures (name, region_id) VALUES (?, ?)",
                (prefecture_name, region_id)
            )
            conn.commit()
            prefecture_id = cursor.lastrowid
        else:
            prefecture_id = prefecture_id[0]

        return prefecture_id
    except sqlite3.Error as e:
        print(f"データベースエラー: {e}")
        return None
    finally:
        conn.close()

def save_area_to_db(area_name, prefecture_id):
    """地域情報をareasテーブルに保存"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO areas (name, prefecture_id) VALUES (?, ?)",
            (area_name, prefecture_id)
        )
        conn.commit()
    except sqlite3.Error as e:
        print(f"データベースエラー: {e}")
    finally:
        conn.close()

def fetch_weather_data(region_code):
    """指定地域コードの天気情報を取得"""
    try:
        url = FORECAST_API_URL.format(region_code=region_code)
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data:
            area_weather = data[0]['timeSeries'][0]['areas']

            for area in area_weather:
                area_name = area["area"]["name"]
                time_defines = data[0]["timeSeries"][0]["timeDefines"]
                weathers = area["weathers"]
                winds = area["winds"]
                waves = area.get("waves", ["なし"] * len(weathers))

                max_length = len(time_defines)
                weathers.extend(["N/A"] * (max_length - len(weathers)))
                winds.extend(["N/A"] * (max_length - len(winds)))
                waves.extend(["なし"] * (max_length - len(waves)))

                weather_details = []
                for time, weather, wind, wave in zip(time_defines, weathers, winds, waves):
                    weather_details.append({
                        "time": time,
                        "weather": weather,
                        "wind": wind,
                        "wave": wave,
                    })

                yield area_name, weather_details
        else:
            print(f"{region_code} の天気情報が取得できませんでした。")
    except requests.exceptions.RequestException as e:
        print(f"APIリクエスト失敗: {e}")
    except json.JSONDecodeError:
        print("JSON解析失敗")

def process_and_save_weather(file_path):
    """JSONから都道府県・天気情報を保存"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for region_code, region_info in data["centers"].items():
                region_name = region_info["name"]
                region_id = save_region_to_db(region_name)

                for child_code in region_info["children"]:
                    if child_code in data["offices"]:
                        child_info = data["offices"][child_code]
                        prefecture_name = child_info["name"]
                        prefecture_id = save_prefecture_to_db(prefecture_name, region_id)

                        # 地域情報を保存
                        area_name = child_info.get("name")
                        save_area_to_db(area_name, prefecture_id)

                        # 天気情報を取得して保存
                        weather_data = fetch_weather_data(child_code)
                        for area_name, weather_details in weather_data:
                            save_weather_to_db(area_name, prefecture_id, weather_details)

    except Exception as e:
        print(f"エラー発生: {e}")

if __name__ == "__main__":
    process_and_save_weather("/Users/terashimaharuki/dsp_2/jma/areas.json")  # 天気情報を保存


APIリクエスト失敗: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
APIリクエスト失敗: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/460040.json


In [276]:
import flet as ft
import sqlite3

# SQLite データベースファイルのパス
DB_FILE = "weather.db"

def fetch_weather_data_from_db():
    """データベースから天気情報を取得"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # 天気情報を取得
        cursor.execute("""
            SELECT p.name AS prefecture, w.date_time, w.weather, w.wind, w.wave
            FROM weather w
            JOIN prefectures p ON w.prefecture_id = p.id
            ORDER BY w.date_time DESC
            LIMIT 10
        """)

        rows = cursor.fetchall()

        # 結果をリスト形式で返す
        return rows

    except sqlite3.Error as e:
        print(f"データベースエラー: {e}")
        return []

    finally:
        conn.close()

def main(page: ft.Page):
    # データベースから天気情報を取得
    weather_data = fetch_weather_data_from_db()

    # データがなければメッセージを表示
    if not weather_data:
        page.add(ft.Text("データがありません"))
        return

    # データをテーブルとして表示
    table = ft.DataTable(
        columns=[
            ft.DataColumn(ft.Text("県名")),
            ft.DataColumn(ft.Text("日時")),
            ft.DataColumn(ft.Text("天気")),
            ft.DataColumn(ft.Text("風")),
            ft.DataColumn(ft.Text("波")),
        ],
        rows=[
            ft.DataRow(cells=[
                ft.DataCell(ft.Text(row[0])),  # 県名
                ft.DataCell(ft.Text(row[1])),  # 日時
                ft.DataCell(ft.Text(row[2])),  # 天気
                ft.DataCell(ft.Text(row[3])),  # 風
                ft.DataCell(ft.Text(row[4])),  # 波
            ])
            for row in weather_data
        ]
    )

    # テーブルをページに追加
    page.add(table)

# Flet アプリケーションの実行
ft.app(target=main)


/Users/terashimaharuki/.pyenv/versions/3.12.7/lib/python3.12/re/__init__.py:260: RuntimeWarning: coroutine 'app_async' was never awaited
  return pattern.translate(_special_chars_map)


RuntimeError: asyncio.run() cannot be called from a running event loop